# AYNEXT use examples

## AYNEXT-DataGen

First, install required dependencies:

In [ ]:
import os
print(os.popen('pip install -r requirements.txt').read())

Launch AYNEXT-DataGen to generate an evaluation dataset from the `WN11.txt` file, which contains a triple in each line. We want to use 20% of the dataset for testing, generating 2 negatives per positive by randomly replacing the source entity of the positive triples, removing relations with less than 10 instances, removing inverses with an overlapping threshold of 0.9. Run in command line:

In [ ]:
import subprocess
%cd DataGen

command = ['python', 'DataGen.py', 
           '--inF', 'WN11.txt', 
           '--outF', 'WN11-dataset', 
           '--minNumRel', '10', 
           '--removeInv', 
           '--change_source_random','2']

for line in subprocess.Popen(command, stdout=subprocess.PIPE, shell=True).communicate()[0].decode('utf-8').splitlines():
    print(line)

Results can be found at `DataGen/WN11-dataset`.

Now, let us suppose that we want to implement a negatives generation strategy that merely replaces the source of the triple with a fictional entity named "foo", and the target with a fictional entity named "bar". We would define the following class that extends the class `NegativesGenerator`:

In [ ]:
from NegativesGenerators import NegativesGenerator

class FooGenerator(NegativesGenerator):
	def __init__(self) -> None:
		self.tc = "FooBar"
	def initialize(self):
		pass
	def generate_negatives(self, positive, number_negatives):
		rel = positive[0]
		negatives = [(rel, "foo", "bar", self.tc) for i in range(number_negatives)]
		return negatives

Next, in order to execute AYNEXT programatically, `generate_datasets` function must be used and provided with every possible param or its default value (in this case, we are using the previously described parameter configuration):

In [ ]:
from DataGen import generate_datasets
negatives_generators = {}
# Define negatives generators to use and number of negatives to generate per relation
negatives_generators[FooGenerator()] = 2

generate_datasets(
    input_file='./WN11.txt', 
    input_format='simpleTriplesReader',
    output_folder='./WN11-dataset', 
	min_num_rel= 10, 
    remove_inverses = True, 
    graph_fraction = 1.0,
    generate_negatives_training = False,
    reach_fraction = 1.0,
    testing_fraction = 0.2,
    validation_fraction = 0.0,
    splitting_technique = 'random',
    pvalue_threshold = 0.05,
    export_gexf = False,
    create_summary = False,
    inverse_threshold = 0.9,
    include_data_prop = True,
    separate_types = True,
    negatives_generators = negatives_generators)

Results can be found at `DataGen/WN11-dataset`.

## AYNEXT-ResTest

ResTest can only be executed by command line. Let us suppose that we want to evaluate the performance of three link prediction techniques AllYes, AllNo and AllRandom, whose results are stored in `mockup-results.txt`. ResTest input parameters defined at the begining of the file are already configured for. Let us just launch ResTest in a command line:

In [ ]:
%cd ..
command = ['python', 'ResTest/ResTest.py']

for line in subprocess.Popen(command, stdout=subprocess.PIPE, shell=True).communicate()[0].decode('utf-8').splitlines():
    print(line)

Results can be found at `ResTest/mockup-metrics.txt` and `ResTest/mockup-pvalues.txt`.